In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    datar = pd.concat([pd.read_csv('./../data/rawdata/%dr.txt' % year,sep=';') for year in range(2003,2017)])
datar.columns = np.array([str(i).strip().replace(' ','_') for i in datar.columns])

In [4]:
datar = datar[['Tournament_Year','Permanent_Tournament_#','Course_#','Player_Number','Player_Name','Round_Number',
               'Round_Score','End_of_Event_Pos._(text)','Event_Name']]

In [5]:
datar.head(3)

,Tournament_Year,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Round_Score,End_of_Event_Pos._(text),Event_Name
0,2003,16,656,1320,"Estes, Bob",1,66,T6,Mercedes Championships
1,2003,16,656,1361,"Forsman, Dan",1,69,T21,Mercedes Championships
2,2003,16,656,1761,"Mattiace, Len",1,75,24,Mercedes Championships


In [6]:
datar['End_of_Event_Pos._(text)'].value_counts().index.values

array([' CUT', ' T13', '  T9', ' T17', ' T30', ' T24', ' T10', ' T27',
       ' T33', ' T22', ' T21', ' T16', ' T15', ' T18', '  T8', ' T12',
       ' T44', ' T20', '  T6', '   1', ' T42', ' T37', '  T5', ' T14',
       ' T48', ' T34', '  T4', ' T11', ' T39', ' T25', ' T45', ' T29',
       ' T40', ' T32', ' T46', ' T26', '  T7', ' T56', ' T59', ' T28',
       ' T52', ' T50', ' T51', ' T36', ' T19', ' T57', ' T47', ' T31',
       ' T38', ' T58', '  T2', ' T54', ' T63', ' T43', '  T3', ' T41',
       ' T53', ' T35', ' T64', ' T23', ' T61', ' T66', ' T65', ' T49',
       ' T60', ' T62', ' T55', ' T68', ' T69', '   2', ' W/D', ' T67',
       ' T70', ' T72', ' T71', '   3', ' T73', ' T74', '   4', ' T75',
       '  71', '  72', '  73', ' T76', '  74', '   5', '  70', ' T77',
       '  69', '  75', '  76', '  68', '  77', ' T78', '   6', '  78',
       ' T79', '   7', '  79', '  67', '   8', '  66', '   9', ' T80',
       ' DQ', '  80', ' T81', '  65', ' T82', '  64', '  10', '  63',
       

In [7]:
len(datar)

261539

In [8]:
datar = datar[~datar['End_of_Event_Pos._(text)'].isin([' W/D',' DQ'])]

In [9]:
len(datar)

259743

In [10]:
df = datar.groupby(['Tournament_Year','Permanent_Tournament_#','Player_Number'],as_index=False).agg({'Round_Number' : 'count',
                                                                                                'Round_Score' : 'sum'})
df.columns = ['Tournament_Year','Permanent_Tournament_#','Player_Number','Total_Strokes','Total_Rounds']

In [11]:
df.head(3)

,Tournament_Year,Permanent_Tournament_#,Player_Number,Total_Strokes,Total_Rounds
0,2003,1,1014,156,2
1,2003,1,1035,149,2
2,2003,1,1042,146,2


In [12]:
pd.Series(df.Total_Strokes/df.Total_Rounds).describe()

count    82590.000000
mean        71.024715
std          7.356489
min          0.000000
25%         70.000000
50%         71.500000
75%         73.000000
max         88.000000
dtype: float64

In [13]:
df[(df.Total_Strokes/df.Total_Rounds)<60]['Permanent_Tournament_#'].value_counts()

470    814
Name: Permanent_Tournament_#, dtype: int64

In [14]:
datar[datar['Permanent_Tournament_#']==470].head(3)

,Tournament_Year,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Round_Score,End_of_Event_Pos._(text),Event_Name
3522,2003,470,663,6251,"DiMarco, Chris",1,0,T33,World Golf Championships-Accenture Match Play ...
3523,2003,470,663,10821,"Izawa, Toshi",1,0,T9,World Golf Championships-Accenture Match Play ...
3524,2003,470,663,1553,"Huston, John",1,0,T33,World Golf Championships-Accenture Match Play ...


In [15]:
d = {tuple(tup[0:3]):tuple(tup[3:]) if tup[1]!=470 else (np.nan,np.nan) for tup in df.values.tolist()}

In [16]:
{key:d[key] for key in d.keys()[0:2]}

{(2006, 24, 1475): (279, 4), (2010, 2, 21731): (346, 5)}

In [17]:
datar.insert(len(datar.columns),'Strokes_In_Tournament',[0]*len(datar))
datar.insert(len(datar.columns),'Rounds_In_Tournament',[0]*len(datar))

In [18]:
datar['Strokes_In_Tournament'] = [d[tuple(tup)][0] for tup in 
                                  datar[['Tournament_Year','Permanent_Tournament_#','Player_Number']].values.tolist()]
datar['Rounds_In_Tournament'] = [d[tuple(tup)][1] for tup in 
                                  datar[['Tournament_Year','Permanent_Tournament_#','Player_Number']].values.tolist()]

In [20]:
datar.head(3)

,Tournament_Year,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Round_Score,End_of_Event_Pos._(text),Event_Name,Strokes_In_Tournament,Rounds_In_Tournament
0,2003,16,656,1320,"Estes, Bob",1,66,T6,Mercedes Championships,271.0,4.0
1,2003,16,656,1361,"Forsman, Dan",1,69,T21,Mercedes Championships,279.0,4.0
2,2003,16,656,1761,"Mattiace, Len",1,75,24,Mercedes Championships,280.0,4.0


In [26]:
pd.Series(datar[datar.Strokes_In_Tournament.notnull()].Strokes_In_Tournament/
          datar[datar.Strokes_In_Tournament.notnull()].Rounds_In_Tournament).describe()

count    257759.000000
mean         71.260166
std           2.376731
min          63.500000
25%          69.500000
50%          71.000000
75%          72.500000
max          88.000000
dtype: float64

In [27]:
datar.insert(len(datar.columns),'Finishing_Position',[0]*len(datar))

In [56]:
assert 'CUT' not in set([fin.strip() for fin in datar[datar['Permanent_Tournament_#']==470]['End_of_Event_Pos._(text)'].values.astype(str)])

In [67]:
finishing_pos_dict = {}
for (year,tourn),df in datar.groupby(['Tournament_Year','Permanent_Tournament_#']):
    df = df.drop_duplicates('Player_Number')
    if tourn==470:
        finishing_pos  = pd.Series([''.join(c for c in f.strip() if c!='T') 
                         for f in df['End_of_Event_Pos._(text)'].values]).astype(float).rank()
        finishing_pos_dict.update({(year,tourn,player):pos/len(df) for player,pos in zip(df.Player_Number,finishing_pos)})
    else:
        df = df.sort_values(['Rounds_In_Tournament','Strokes_In_Tournament'],ascending=[False,True])
        running_count = 0
        for num_rounds,df_ in df.groupby(['Rounds_In_Tournament'],sort=False):
            finishing_pos = df_.Strokes_In_Tournament.rank()
            finishing_pos_dict.update({(year,tourn,player):pos/len(df) for player,pos in zip(df_.Player_Number,finishing_pos)})
            running_count += len(df_)

In [69]:
{key:finishing_pos_dict[key] for key in finishing_pos_dict.keys()[0:2]}

{(2006, 24, 1475): 0.32236842105263158, (2010, 2, 21731): 0.42857142857142855}

In [70]:
datar['Finishing_Position'] = [finishing_pos_dict[tuple(tup)] 
                               for tup in datar[['Tournament_Year','Permanent_Tournament_#','Player_Number']].values.tolist()]

In [71]:
datar.Finishing_Position.describe()

count    259743.000000
mean          0.277149
std           0.179741
min           0.005587
25%           0.131410
50%           0.262069
75%           0.397436
max           1.000000
Name: Finishing_Position, dtype: float64

In [72]:
datar.to_csv('./../data/round.csv',index=False)